# Plot Hiking Trail on Map

In [ ]:
# install library
!pip install gpxpy

In [ ]:
import gpxpy
import folium
import pandas as pd
import numpy as np
import branca.colormap as cm

## Load data from GPX file

In [ ]:
gpx_file = "Old Manali - Wayland Viewpoint Trail.gpx"

with open(gpx_file, 'r') as f:
        gpx = gpxpy.parse(f)

data = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            data.append({
                'longitude': point.longitude,
                'latitude': point.latitude,
                'elevation': point.elevation
            })

# Create DataFrame
df = pd.DataFrame(data)
df = df.iloc[:round(df.shape[0]/2)]

In [ ]:
df.head()

In [ ]:
df.shape

## Plot hiking trail on a map

In [ ]:
# Create a Folium Map centered around the start of the trail
start_lat = df['latitude'].iloc[0]
start_lon = df['longitude'].iloc[0]
m = folium.Map(location=[start_lat, start_lon], zoom_start=13)

# Add markers for the start and end points (optional but helpful)

# Start marker
folium.Marker(
    location=[df['latitude'].iloc[0], df['longitude'].iloc[0]],
    popup='Start of Trail',
    icon=folium.Icon(color='green')
).add_to(m)

# End marker
folium.Marker(
    location=[df['latitude'].iloc[-1], df['longitude'].iloc[-1]],
    popup='End of Trail',
    icon=folium.Icon(color='red')
).add_to(m)

# Extract the coordinates for the trail line
trail_coords = df[['latitude', 'longitude']].values.tolist()

# Add the trail line to the map
folium.PolyLine(
    locations=trail_coords,
    color='blue',
    weight=2.5,
    opacity=1
).add_to(m)

# 6. Display the map
display(m)

## Plot trail with elevation data

In [ ]:
start_lat = df['latitude'].iloc[0]
start_lon = df['longitude'].iloc[0]
m = folium.Map(location=[start_lat, start_lon], zoom_start=13)

In [ ]:
# Add markers
folium.Marker(
        location=[df['latitude'].iloc[0], df['longitude'].iloc[0]],
        popup='Start of Trail (Elevation: {})'.format(df['elevation'].iloc[0]),
        icon=folium.Icon(color='green')
    ).add_to(m)

folium.Marker(
    location=[df['latitude'].iloc[-1], df['longitude'].iloc[-1]],
    popup='End of Trail (Elevation: {})'.format(df['elevation'].iloc[-1]),
    icon=folium.Icon(color='red')
).add_to(m)

In [ ]:
min_elevation = df['elevation'].min()
max_elevation = df['elevation'].max()

# Create a colormap (e.g., Viridis)
# You can choose different colormaps from branca.colormap
colormap = cm.LinearColormap(
    ['blue', 'green', 'yellow', 'red'],  # Colors for low to high elevation
    vmin=min_elevation,
    vmax=max_elevation
)

# Add the colormap legend to the map
#colormap.add_to(m, position='topright')
m.add_child(colormap)

# Iterate through the trail points and draw colored segments
for i in range(len(df) - 1):
    # Get the coordinates of the current and next point
    start_coords = [df['latitude'].iloc[i], df['longitude'].iloc[i]]
    end_coords = [df['latitude'].iloc[i+1], df['longitude'].iloc[i+1]]

    # Get the elevation at the starting point of the segment
    segment_elevation = df['elevation'].iloc[i]

    # Get the color for this elevation from the colormap
    segment_color = colormap(segment_elevation)

    # Draw a PolyLine segment with the calculated color
    folium.PolyLine(
        locations=[start_coords, end_coords],
        color=segment_color,
        weight=5.5,
        opacity=1,
        popup=f"Elevation: {segment_elevation:.1f}" # Optional popup for segment elevation
    ).add_to(m)

display(m)